# API Usage Demo

This notebook demonstrates how to use the AI-Based Transaction Failure Prediction API.

In [2]:
import sys, os
project_root = os.path.dirname(os.getcwd())  # Go up one level from notebooks/
if project_root not in sys.path:
    sys.path.insert(0, project_root)
print(f"Project root added to path: {project_root}")

Project root added to path: d:\AI_Semester_Project\failure-transaction-algo


In [3]:
import pandas as pd
import numpy as np
import requests
import json
from datetime import datetime

# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd()))
sys.path.insert(0, project_root)

# Import our modules
from src.simulator.transaction_simulator import TransactionSimulator

# API configuration
API_BASE_URL = "http://localhost:8000"
print(f"Using API at: {API_BASE_URL}")

Using API at: http://localhost:8000


## Start the API Server

Before using the API, we need to start the server. In a real scenario, you would run `uvicorn src.api.main:app --host 0.0.0.0 --port 8000` from the command line.

In [4]:
# Check if the API is running by querying the health endpoint
try:
    response = requests.get(f"{API_BASE_URL}/health")
    if response.status_code == 200:
        health_data = response.json()
        print(f"API Health: {health_data}")
    else:
        print(f"API not responding. Status code: {response.status_code}")
        print("Please start the API server with: uvicorn src.api.main:app --host 0.0.0.0 --port 8000")
except requests.exceptions.ConnectionError:
    print("Connection error: API server is not running.")
    print("Please start the API server with: uvicorn src.api.main:app --host 0.0.0.0 --port 8000")

API Health: {'status': 'healthy', 'model_loaded': True, 'scaler_loaded': True}


## Generate Test Transactions

Generate some test transactions to use with the API.

In [5]:
# Generate test transactions
simulator = TransactionSimulator(failure_rate=0.15)
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 12, 31)

# Generate 5 sample transactions
sample_transactions = []
for i in range(5):
    transaction = simulator.generate_transaction(start_date, end_date)
    sample_transactions.append(transaction)

# Convert to DataFrame for better visualization
sample_df = pd.DataFrame(sample_transactions)
print(f"Generated {len(sample_df)} sample transactions")
sample_df[['transaction_amount', 'account_balance', 'location', 'transaction_failure', 'failure_type']]

Generated 5 sample transactions


,transaction_amount,account_balance,location,transaction_failure,failure_type
0,57.590572,90.380634,local,1,insufficient_funds
1,24.682916,16.930594,high_risk_location_1,0,success
2,1.750474,43.718484,high_risk_location_2,1,timeout
3,21.061812,67.298895,local,0,success
4,15.286171,509.092160,high_risk_location_1,0,success


## Single Transaction Prediction

Test the API with a single transaction prediction.

In [10]:
# Take the first transaction for testing
test_transaction = sample_df.iloc[2]

# Prepare the request data
transaction_data = {
    "transaction_amount": float(test_transaction['transaction_amount']),
    "account_balance": float(test_transaction['account_balance']),
    "time_of_day": int(test_transaction['time_of_day']),
    "day_of_week": int(test_transaction['day_of_week']),
    "transaction_type": test_transaction['transaction_type'],
    "location": test_transaction['location'],
    "merchant_category": test_transaction['merchant_category'],
    "location_risk_score": float(test_transaction['location_risk_score']),
    "historical_failure_rate": float(test_transaction['historical_failure_rate'])
}

print("Test transaction data:")
for key, value in transaction_data.items():
    print(f"  {key}: {value}")

# Make the API request
try:
    response = requests.post(f"{API_BASE_URL}/predict", json=transaction_data)
    
    if response.status_code == 200:
        result = response.json()
        print(f"\nPrediction result:")
        print(f"  Failure probability: {result['failure_probability']:.4f}")
        print(f"  Prediction: {'Failure' if result['prediction'] == 1 else 'Success'}")
        print(f"  Failure reason: {result['failure_reason']}")
        
        # Compare with actual value
        actual_failure = test_transaction['transaction_failure']
        print(f"  Actual outcome: {'Failure' if actual_failure == 1 else 'Success'}")
        print(f"  Prediction {'correct' if result['prediction'] == actual_failure else 'incorrect'}")
    else:
        print(f"API request failed with status code: {response.status_code}")
        print(f"Error: {response.text}")
        
except requests.exceptions.ConnectionError:
    print("\nCannot connect to API. Please make sure the server is running.")

Test transaction data:
  transaction_amount: 1.750473640513014
  account_balance: 43.71848424954305
  time_of_day: 6
  day_of_week: 3
  transaction_type: credit
  location: high_risk_location_2
  merchant_category: grocery
  location_risk_score: 0.3115718583523256
  historical_failure_rate: 0.20100872072832193

Prediction result:
  Failure probability: 0.1607
  Prediction: Success
  Failure reason: None
  Actual outcome: Failure
  Prediction incorrect


## Batch Transaction Prediction

Test the API with multiple transactions at once.

In [7]:
# Prepare batch request data
batch_transactions = []
for i in range(min(5, len(sample_df))):  # Take up to 5 transactions
    row = sample_df.iloc[i]
    transaction_data = {
        "transaction_amount": float(row['transaction_amount']),
        "account_balance": float(row['account_balance']),
        "time_of_day": int(row['time_of_day']),
        "day_of_week": int(row['day_of_week']),
        "transaction_type": row['transaction_type'],
        "location": row['location'],
        "merchant_category": row['merchant_category'],
        "location_risk_score": float(row['location_risk_score']),
        "historical_failure_rate": float(row['historical_failure_rate'])
    }
    batch_transactions.append(transaction_data)

batch_request = {"transactions": batch_transactions}

# Make the batch API request
try:
    response = requests.post(f"{API_BASE_URL}/predict_batch", json=batch_request)
    
    if response.status_code == 200:
        results = response.json()
        print(f"Batch prediction results for {len(results)} transactions:")
        
        for i, result in enumerate(results):
            actual = sample_df.iloc[i]['transaction_failure']
            correctness = 'correct' if result['prediction'] == actual else 'incorrect'
            print(f"  Transaction {i+1}: Prob={result['failure_probability']:.3f}, "
                  f"Pred={'Failure' if result['prediction'] == 1 else 'Success'}, "
                  f"Actual={'Failure' if actual == 1 else 'Success'}, "
                  f"{correctness}")
    else:
        print(f"Batch API request failed with status code: {response.status_code}")
        print(f"Error: {response.text}")
        
except requests.exceptions.ConnectionError:
    print("\nCannot connect to API. Please make sure the server is running.")

Batch prediction results for 5 transactions:
  Transaction 1: Prob=0.146, Pred=Success, Actual=Failure, incorrect
  Transaction 2: Prob=0.166, Pred=Success, Actual=Success, correct
  Transaction 3: Prob=0.161, Pred=Success, Actual=Failure, incorrect
  Transaction 4: Prob=0.117, Pred=Success, Actual=Success, correct
  Transaction 5: Prob=0.482, Pred=Success, Actual=Success, correct


## Get Model Information

Retrieve information about the loaded model.

In [8]:
# Get model information
try:
    response = requests.get(f"{API_BASE_URL}/model_info")
    
    if response.status_code == 200:
        model_info = response.json()
        print("Model Information:")
        for key, value in model_info.items():
            print(f"  {key}: {value}")
    else:
        print(f"Model info request failed with status code: {response.status_code}")
        print(f"Error: {response.text}")
        
except requests.exceptions.ConnectionError:
    print("\nCannot connect to API. Please make sure the server is running.")

Model Information:
  model_type: GaussianNB
  model_loaded: True
  scaler_loaded: True
  features_count: 9
  feature_info: {}


## API Documentation

The API also provides automatic documentation at `/docs` and `/redoc` endpoints.

In [9]:
print(f"API Documentation:")
print(f"  Interactive docs: {API_BASE_URL}/docs")
print(f"  Alternative docs: {API_BASE_URL}/redoc")

API Documentation:
  Interactive docs: http://localhost:8000/docs
  Alternative docs: http://localhost:8000/redoc


## Summary

We have demonstrated how to use the AI-Based Transaction Failure Prediction API:
1. Verified that the API server is running
2. Generated test transactions
3. Made single transaction predictions
4. Made batch transaction predictions
5. Retrieved model information
6. Accessed API documentation

The API provides a RESTful interface for making transaction failure predictions using the trained ML models. It accepts transaction details and returns the probability of failure along with a prediction.